<a href="https://colab.research.google.com/github/sebinbyju007/Intelligent_QA_using_Deep_Learning/blob/main/Copy_of_Final_yr_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = pd.read_csv('drive/MyDrive/train.csv.zip')

In [ ]:
!cd 

/content


# New Section

In [ ]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
embedding_index={}

In [ ]:
glove_file = open('drive/MyDrive/glove.6B.300d.txt',encoding="utf8")

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!unzip glove*.zip

In [ ]:
for line in glove_file:
    values = line.split()
    embedding_index[values[0]] = np.asarray(values[1:],dtype='float32')
glove_file.close()    

In [ ]:
data.shape

(404290, 6)

In [ ]:
data=data.dropna()
data.shape

(404287, 6)

In [ ]:
qstin1_list = list(data['question1'])
qstin2_list = list(data['question2'])
texts = qstin1_list+qstin2_list

In [ ]:
len(texts)

808574

In [ ]:
len(embedding_index)

400000

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

In [ ]:
embedding_dimension=300

In [ ]:
embedding_matrix = np.zeros((len(word_index)+1,embedding_dimension))

In [ ]:
len(word_index)

95595

In [ ]:
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector[:embedding_dimension]

In [ ]:
embedding_matrix.shape

(95596, 300)

In [ ]:
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

In [ ]:
embedding_layer = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights = [embedding_matrix],input_length = 30)

In [ ]:
X_ques1 = tokenizer.texts_to_sequences(qstin1_list)
X_ques1 = pad_sequences(X_ques1, maxlen=30,padding='post')

In [ ]:
X_ques2 = tokenizer.texts_to_sequences(qstin2_list)
X_ques2 = pad_sequences(X_ques2,maxlen = 30,padding='post')

In [ ]:
data['X_ques1'] = list(X_ques1)
data['X_ques2'] = list(X_ques2)

In [ ]:
print(X_ques1[7])
print(texts[7])
word_index['geologist']

[    4    13     5    24     6    42 25874     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
How can I be a good geologist?


25874

In [ ]:
data.head(8)

,id,qid1,qid2,question1,question2,is_duplicate,X_ques1,X_ques2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[2, 3, 1, 1222, 57, 1222, 2581, 7, 576, 8, 763...","[2, 3, 1, 1222, 57, 1222, 2581, 7, 576, 8, 763..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[2, 3, 1, 559, 10, 14300, 13598, 5, 21311, 456...","[2, 43, 182, 25, 1, 82, 237, 11296, 1, 14300, ..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[4, 13, 5, 217, 1, 440, 10, 17, 361, 1827, 200...","[4, 13, 361, 440, 24, 3338, 57, 1344, 219, 109..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[16, 72, 5, 2774, 312, 2757, 4, 13, 5, 649, 19...","[87, 1, 4170, 37, 230, 2234, 1343, 230, 3, 245..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[23, 49, 7131, 8, 231, 35496, 1891, 2047, 1057...","[23, 1945, 43, 1242, 8, 2047, 231, 0, 0, 0, 0,..."
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[2371, 5, 72, 6, 9925, 940, 4451, 813, 12, 445...","[144, 6, 4894, 9925, 940, 813, 12, 9297, 8, 99..."
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,"[31, 5, 126, 31238, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2, 2355, 77409, 1530, 12, 1068, 33, 122, 12, ..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[4, 13, 5, 24, 6, 42, 25874, 0, 0, 0, 0, 0, 0,...","[2, 31, 5, 9, 7, 24, 6, 338, 25874, 0, 0, 0, 0..."


In [ ]:
labels = np.asarray(data['is_duplicate'])

In [ ]:
len(labels)

404287

In [ ]:
#functional API
from keras.models import Sequential

from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPool1D, Dense, Flatten, Dropout, Conv2D
from keras.layers import Bidirectional
import keras.backend as K
from keras.layers import Layer

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPool1D(5))
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPool1D(5))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
#model.add(attention())
model.add(Flatten())
model.add(Dense(36))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['acc'])
model.summary()

ValueError: ignored

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128,3,activation='relu',input_shape=(30,30)))
model.add(MaxPool1D(5))
model.add(Conv1D(128,3,activation='relu'))
model.add(MaxPool1D(2))
# model.add(Dropout(0.25))
# model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
# model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2)))
#model.add(attention())
model.add(Flatten())
model.add(Dense(36))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           9559600   
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 28, 128)           38528     
_________________________________________________________________
max_pooling1d_71 (MaxPooling (None, 5, 128)            0         
_________________________________________________________________
conv1d_74 (Conv1D)           (None, 3, 128)            49280     
_________________________________________________________________
max_pooling1d_72 (MaxPooling (None, 1, 128)            0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 36)              

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(64,3,activation='relu',input_shape=X_ques1.shape))
model.add(MaxPool1D(3,3))
model.add(Conv1D(32,3,activation='relu'))
model.add(MaxPool1D(3,3))
model.add(Dropout(0.05))
model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
model.add(Bidirectional(LSTM(100,activation='tanh',dropout=0.2,recurrent_dropout=0.2)))
#model.add(attention())
model.add(Flatten())
model.add(Dense(36,activation='relu'))
model.add(Dense(1,activation='softmax'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['acc'])
model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           9559600   
_________________________________________________________________
conv1d_71 (Conv1D)           (None, 28, 64)            19264     
_________________________________________________________________
max_pooling1d_69 (MaxPooling (None, 9, 64)             0         
_________________________________________________________________
conv1d_72 (Conv1D)           (None, 7, 32)             6176      
_________________________________________________________________
max_pooling1d_70 (MaxPooling (None, 2, 32)             0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 2, 32)             0         
_________________________________________________________________
bidirectional_54 (Bidirectio (None, 2, 200)          

In [ ]:
#19/04/21
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights = [embedding_matrix],input_length = 30))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional((LSTM(100))))
# model.add(attention())
model.add(Flatten())
model.add(Dense(200,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 100)           9559600   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 32)            9632      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 15, 200)           106400    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               240800    
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)              

In [ ]:
#20/04/21
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights = [embedding_matrix],input_length = 30))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional((LSTM(100))))
# model.add(attention())
model.add(Flatten())
model.add(Dense(200,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           9559600   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 32)            9632      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 32)            3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7, 32)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 200)            106400    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)              

In [ ]:
#21/04/21
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights = [embedding_matrix],input_length = 30))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Dropout(0.07))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Dropout(0.07))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional((LSTM(100))))
model.add(Dropout(0.07))
# model.add(attention())
model.add(Flatten())
model.add(Dense(200,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           28678800  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 32)            28832     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 32)            3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7, 32)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 32)            

In [ ]:
filepath="weights_best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto',save_weights_only=False)
callbacks_list = [checkpoint]

In [ ]:
# model = Sequential()
# model.add(embedding_layer)
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.summary()

In [ ]:
train_len = int(len(X_ques1)*0.2)

In [ ]:
train_len

80857

In [ ]:
model.fit(list([X_ques1[0:train_len],X_ques2[0:train_len]]), y=labels[0:train_len], epochs = 3, batch_size=256,verbose = 1,callbacks = callbacks_list, shuffle=True,validation_data =([X_ques1[train_len:train_len+1000],X_ques2[train_len:train_len+1000]],labels[train_len:train_len+1000]))

Epoch 1/3
316/316 [==============================] - 139s 439ms/step - loss: 0.4725 - accuracy: 0.7731 - val_loss: 0.5236 - val_accuracy: 0.7430

Epoch 00001: saving model to weights_best.hdf5
Epoch 2/3
316/316 [==============================] - 138s 436ms/step - loss: 0.3966 - accuracy: 0.8150 - val_loss: 0.5496 - val_accuracy: 0.7330

Epoch 00002: saving model to weights_best.hdf5
Epoch 3/3
316/316 [==============================] - 137s 434ms/step - loss: 0.3251 - accuracy: 0.8530 - val_loss: 0.6177 - val_accuracy: 0.7440

Epoch 00003: saving model to weights_best.hdf5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save('sample_checkpoint_all_false.h5')

In [ ]:
model.evaluate([X_ques1[train_len:],X_ques2[train_len:]],labels[train_len:])

10108/10108 [==============================] - 92s 9ms/step - loss: 0.6250 - accuracy: 0.7340


[0.6249841451644897, 0.7339826226234436]

In [ ]:
model.predict([X_ques1[0:1],X_ques2[0:1]])

array([[0.31576172]], dtype=float32)

In [ ]:
dup_data = data[data['is_duplicate']==1]

In [ ]:
X_dup_ques1 = dup_data['X_ques1']
X_dup_ques2 = dup_data['X_ques2']

In [ ]:
dup_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,X_ques1,X_ques2
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[2371, 5, 72, 6, 9925, 940, 4451, 813, 12, 445...","[144, 6, 4894, 9925, 940, 813, 12, 9297, 8, 99..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[4, 13, 5, 24, 6, 42, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 31, 5, 9, 7, 24, 6, 338, 0, 0, 0, 0, 0, 0,..."
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1,"[4, 9, 5, 223, 12, 87, 17, 286, 1727, 0, 0, 0,...","[4, 13, 5, 165, 81, 17, 286, 1727, 0, 0, 0, 0,..."
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1,"[2, 13, 52, 609, 632, 7, 71, 0, 0, 0, 0, 0, 0,...","[4, 13, 15, 52, 609, 632, 7, 71, 0, 0, 0, 0, 0..."
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1,"[2, 59, 34, 94, 1403, 299, 39, 0, 0, 0, 0, 0, ...","[2, 59, 34, 94, 1403, 299, 0, 0, 0, 0, 0, 0, 0..."


In [ ]:
X_dup_ques1[5]

array([2371,    5,   72,    6, 9925,  940, 4451,  813,   12, 4451, 5037,
          2,   21,   28,  238,   46,   60,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
arrk=model.predict([X_ques1[400000:400100],X_ques2[400000:400100]])

In [ ]:
arr_v = data['is_duplicate'][400000:400100]
k=(zip(arrk,arr_v))
for i,j in k:
  print(1 if i[0]>=0.5 else 0,j)

1 1
1 1
1 1
0 0
0 0
0 0
0 1
1 0
0 1
0 1
0 0
1 1
1 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
1 1
0 0
1 1
1 0
1 1
1 1
0 0
0 0
0 0
1 0
0 0
1 1
1 1
0 0
0 0
0 0
1 1
0 0
0 0
1 1
1 1
0 0
0 0
1 0
0 1
0 1
0 0
1 1
1 1
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 1
1 1
1 1
1 1
0 1
1 1
1 1
1 0
1 0
0 1
0 0
0 0
0 0
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
1 0
1 1
1 1
0 0
0 0
0 0
0 0
1 1
0 0
1 0
1 1
0 0
1 1
0 0
1 0
0 0
1 1
0 1


In [ ]:
(X_ques1[0:1])

array([[   2,    3,    1, 1222,   57, 1222, 2581,    7,  576,    8,  763,
         383,    8,   35,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

In [ ]:
X_ques1

array([[ 2,  3,  1, ...,  0,  0,  0],
       [ 2,  3,  1, ...,  0,  0,  0],
       [ 4, 13,  5, ...,  0,  0,  0],
       ...,
       [ 2,  3, 49, ...,  0,  0,  0],
       [ 2,  3,  1, ...,  0,  0,  0],
       [ 2,  3, 39, ...,  0,  0,  0]])

In [ ]:
X_ques1[0:train_len]

array([[  2,   3,   1, ...,   0,   0,   0],
       [  2,   3,   1, ...,   0,   0,   0],
       [  4,  13,   5, ...,   0,   0,   0],
       ...,
       [  4,  36, 159, ...,   0,   0,   0],
       [  3,   1, 244, ...,   0,   0,   0],
       [  2, 433,  10, ...,   0,   0,   0]])